## Завантаження бібліотек

In [ ]:
!pip install openai
!pip install gradio openai -q
!pip install --upgrade google-api-python-client google-auth-httplib2 gradio openai
!pip install google-auth-oauthlib==0.4.6

  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.3
    Uninstalling google-auth-oauthlib-1.2.3:
      Successfully uninstalled google-auth-oauthlib-1.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.30.0 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.


## Імпорт та налаштування клієнта

In [ ]:
import os
import json
import base64
import gradio as gr
from email.mime.text import MIMEText
from datetime import datetime, timedelta
from openai import OpenAI

# Бібліотеки Google
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [ ]:
os.environ["OPENAI_API_KEY"] = "..."

In [ ]:
CREDENTIALS_PATH = '/content/drive/MyDrive/Lab67/credentials.json'
TOKEN_PATH = '/content/token.json'

SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/gmail.send'
]


In [ ]:
def force_login():
    if not os.path.exists(CREDENTIALS_PATH):
        print(f"❌ ПОМИЛКА: Файл не знайдено за адресою: {CREDENTIALS_PATH}")
        print("Перевір шлях або завантаж файл credentials.json у папку зліва.")
        return

    print("🚀 Починаємо авторизацію...")
    try:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
        # Цей метод виведе посилання в консоль
        creds = flow.run_console()

        # Зберігаємо токен, щоб агент міг ним користуватися
        with open('/content/token.json', 'w') as token:
            token.write(creds.to_json())

        print("\n✅ УСПІХ! Авторизацію пройдено. Файл token.json створено.")

    except Exception as e:
        print(f"\n❌ Помилка під час входу: {e}")

force_login()

🚀 Починаємо авторизацію...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=139159002973-gku2em3aeqtdhi6einhmpv5ioacrbnqd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=LMjpnkkUaGrkKRSPAdMBxQfTgIsmTq&prompt=consent&access_type=offline
Enter the authorization code: 4/1ATX87lOhQlkIdPFHNPjpvuJDb6pfbdEv6Hm4_O277ZDoPDJ5L8W0ON66wnw

✅ УСПІХ! Авторизацію пройдено. Файл token.json створено.
Тепер можеш запускати код Агента.


In [ ]:
# --- АВТОРИЗАЦІЯ GOOGLE ---
def get_service(service_name, version):
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH)
        return build(service_name, version, credentials=creds)
    else:
        print("❌ Токен не знайдено! Спочатку пройди авторизацію.")
        return None

## Створення Інструментів (Tools)

--- БЛОК 1: ІНСТРУМЕНТИ МАГАЗИНУ (E-commerce) ---

In [ ]:
def get_item_price(item_name):
    """Повертає ціну товару."""
    print(f"   [SHOP] 🔍 Шукаю: {item_name}")

    catalog = {"iphone 15": 900, "macbook air": 1200, "sony headphones": 200, "tesla model 3": 35000}

    price = catalog.get(item_name.lower())

    if price:
        return json.dumps({"item": item_name, "price": price, "currency": "USD"})
    return json.dumps({"error": "Item not found"})

In [ ]:
def calculate_shipping(destination, price):
    """Рахує доставку."""
    print(f"   [LOGISTICS] 🚚 Доставка в {destination}")
    base = 50
    insurance = int(price) * 0.05
    return json.dumps({"destination": destination, "total_shipping": base + insurance, "currency": "USD"})

--- БЛОК 2: ОФІСНІ ІНСТРУМЕНТИ (Productivity) ---

In [ ]:
# 1. Google Calendar (РЕАЛЬНИЙ)
def book_meeting(topic, datetime_str, participants):
    print(f"   [CALENDAR] Спроба створити подію: {topic} на {datetime_str}")
    service = get_service('calendar', 'v3')
    if not service: return "Error: No auth token"

    try:
        # Конвертуємо час
        start_dt = datetime.fromisoformat(datetime_str)
        end_dt = start_dt + timedelta(minutes=60)

        event = {
            'summary': topic,
            'description': f"Учасники: {participants}",
            'start': {'dateTime': start_dt.isoformat(), 'timeZone': 'Europe/Kiev'},
            'end': {'dateTime': end_dt.isoformat(), 'timeZone': 'Europe/Kiev'},
        }
        event = service.events().insert(calendarId='primary', body=event).execute()
        return json.dumps({"status": "success", "link": event.get('htmlLink')})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

In [ ]:
# 2. Gmail (РЕАЛЬНИЙ)
def send_email(recipient, subject, body):
    print(f"   [GMAIL] Спроба відправити лист: {recipient}")
    service = get_service('gmail', 'v1')
    if not service: return "Error: No auth token"

    try:
        message = MIMEText(body)
        message['to'] = recipient
        message['subject'] = subject
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        sent = service.users().messages().send(userId='me', body={'raw': raw}).execute()
        return json.dumps({"status": "sent", "id": sent['id']})
    except Exception as e:
        return json.dumps({"status": "error", "message": str(e)})

--- БЛОК 3: РЕЄСТРАЦІЯ ІНСТРУМЕНТІВ (SCHEMA) ---

In [ ]:
# --- СХЕМА ДЛЯ OPENAI ---

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_item_price",
            "description": "Дізнатися ціну товару.",
            "parameters": {
                "type": "object",
                "properties": {"item_name": {"type": "string"}},
                "required": ["item_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_shipping",
            "description": "Порахувати доставку.",
            "parameters": {
                "type": "object",
                "properties": {"destination": {"type": "string"}, "price": {"type": "integer"}},
                "required": ["destination", "price"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_meeting",
            "description": "Забронювати зустріч у Google Calendar. Час повинен бути у форматі ISO (YYYY-MM-DDTHH:MM:SS).",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string"},
                    "datetime_str": {"type": "string", "description": "Start time in ISO format e.g. 2023-10-10T14:00:00"},
                    "participants": {"type": "string"}
                },
                "required": ["topic", "datetime_str"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Відправити email через Gmail.",
            "parameters": {
                "type": "object",
                "properties": {
                    "recipient": {"type": "string"},
                    "subject": {"type": "string"},
                    "body": {"type": "string"}
                },
                "required": ["recipient", "subject", "body"]
            }
        }
    }
]

In [ ]:
available_functions = {
    "get_item_price": get_item_price,
    "calculate_shipping": calculate_shipping,
    "book_meeting": book_meeting,
    "send_email": send_email,
}

## Логіка Агента (Orchestrator)

In [ ]:
# --- ЛОГІКА АГЕНТА ---

client = OpenAI()

current_time = datetime.now().strftime("%Y-%m-%d %H:%M")

system_prompt = {
    "role": "system",
    "content": f"Ти — бізнес-асистент. Зараз {current_time}. Ти маєш доступ до реального Google Calendar та Gmail. "
               "Коли користувач просить запланувати зустріч, перетвори час у формат ISO YYYY-MM-DDTHH:MM:SS."
}

messages_history = [system_prompt]

In [ ]:
def chat_logic(user_input, history):
    global messages_history
    messages_history.append({"role": "user", "content": user_input})

    # 1. Запит до GPT
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages_history, tools=tools)
    msg = response.choices[0].message

    # 2. Якщо GPT хоче викликати інструмент
    if msg.tool_calls:
        messages_history.append(msg)
        for tool in msg.tool_calls:
            print(f"🤖 Виклик функції: {tool.function.name}")
            func = available_functions[tool.function.name]
            args = json.loads(tool.function.arguments)

            # Передача аргументів
            if tool.function.name == "book_meeting":
                res = func(args.get("topic"), args.get("datetime_str"), args.get("participants", ""))
            elif tool.function.name == "send_email":
                res = func(args.get("recipient"), args.get("subject"), args.get("body"))
            elif tool.function.name == "get_item_price":
                res = func(args.get("item_name"))
            elif tool.function.name == "calculate_shipping":
                res = func(args.get("destination"), args.get("price"))

            messages_history.append({"tool_call_id": tool.id, "role": "tool", "name": tool.function.name, "content": str(res)})

        # 3. Фінальна відповідь GPT
        final = client.chat.completions.create(model="gpt-4o-mini", messages=messages_history)
        return final.choices[0].message.content

    return msg.content

In [ ]:
def chat_logic_enhanced(user_input, history):
    global messages_history
    messages_history.append({"role": "user", "content": user_input})

    # Логування для UI
    debug_info = []

    # 1. Запит
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages_history, tools=tools)
    msg = response.choices[0].message

    if msg.tool_calls:
        messages_history.append(msg)
        for tool in msg.tool_calls:
            # Збираємо інфу для відображення
            debug_info.append(f"🛠️ Tool Call: {tool.function.name}\nArgs: {tool.function.arguments}")

            func = available_functions[tool.function.name]
            args = json.loads(tool.function.arguments)

            # Виконання
            if tool.function.name == "book_meeting":
                res = func(args.get("topic"), args.get("datetime_str"), args.get("participants", ""))
            elif tool.function.name == "send_email":
                res = func(args.get("recipient"), args.get("subject"), args.get("body"))
            elif tool.function.name == "get_item_price":
                res = func(args.get("item_name"))
            elif tool.function.name == "calculate_shipping":
                res = func(args.get("destination"), args.get("price"))

            messages_history.append({"tool_call_id": tool.id, "role": "tool", "name": tool.function.name, "content": str(res)})
            debug_info.append(f"✅ Result: {str(res)}")

        final = client.chat.completions.create(model="gpt-4o-mini", messages=messages_history)
        return final.choices[0].message.content, "\n\n".join(debug_info)

    return msg.content, "No tools used."

## Комплексний Тест (Cross-Domain)

In [ ]:
# --- ЗАПУСК GRADIO ---
demo = gr.ChatInterface(
    fn=chat_logic,
    title="🤖 REAL Agent (Gmail + Calendar)",
    description="Агент має доступ до вашого Google Calendar та Gmail."
)

In [ ]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://458d560e4cc9e26baf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🤖 Виклик функції: book_meeting
   [CALENDAR] Спроба створити подію: Захист Лаби на 2025-12-13T10:00:00
🤖 Виклик функції: send_email
   [GMAIL] Спроба відправити лист: mykhailo.matsyshyn.oi.2022@lpnu.ua
🤖 Виклик функції: get_item_price
   [SHOP] 🔍 Шукаю: iPhone 15
🤖 Виклик функції: get_item_price
   [SHOP] 🔍 Шукаю: iPhone 15
🤖 Виклик функції: calculate_shipping
   [LOGISTICS] 🚚 Доставка в Львів
🤖 Виклик функції: send_email
   [GMAIL] Спроба відправити лист: mykhailo.matsyshyn.oi.2022@lpnu.ua
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://458d560e4cc9e26baf.gradio.live


In [ ]:
# Створюємо інтерфейс з додатковим блоком
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🤖 AI Agent Dashboard")
    gr.Markdown("Agent connected to: **Google Calendar, Gmail, Mock Store**")

    with gr.Row():
        chatbot = gr.Chatbot(label="Chat")
        # Додаткове вікно для технічних логів
        debug_output = gr.Textbox(label="🔍 Internal Logs (Agent Thoughts)", lines=10, interactive=False)

    msg = gr.Textbox(label="Type your message here...")
    clear = gr.Button("Clear")

    def user_msg(user_message, history):
        return "", history + [[user_message, None]]

    def bot_msg(history):
        user_message = history[-1][0]
        bot_response, debug_log = chat_logic_enhanced(user_message, history) # Викликаємо нову функцію
        history[-1][1] = bot_response
        return history, debug_log

    msg.submit(user_msg, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_msg, chatbot, [chatbot, debug_output]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)

/tmp/ipython-input-3493002528.py:2: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme="soft") as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://91935fa85e2c387ac0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://91935fa85e2c387ac0.gradio.live


In [ ]:
import pandas as pd

# Велика база даних (50+ товарів)
data = {
    "item_name": [
        # --- Apple ---
        "iPhone 15 128GB", "iPhone 15 Plus 128GB", "iPhone 15 Pro 128GB", "iPhone 15 Pro Max 256GB", "iPhone 15 Pro Max 1TB",
        "iPhone 14 128GB", "iPhone 14 Plus", "iPhone 13", "iPhone SE (2022)",
        "iPad Pro 11 M4", "iPad Air M2", "MacBook Air M2", "MacBook Air M3", "MacBook Pro 14 M3",
        "AirPods Pro 2", "AirPods Max",

        # --- Samsung ---
        "Samsung Galaxy S24", "Samsung Galaxy S24 Plus", "Samsung Galaxy S24 Ultra 256GB", "Samsung Galaxy S24 Ultra 1TB",
        "Samsung Galaxy S23 FE", "Samsung Galaxy A55", "Samsung Galaxy A35",
        "Samsung Galaxy Z Fold 5", "Samsung Galaxy Z Flip 5",
        "Samsung Galaxy Tab S9 Ultra", "Samsung Galaxy Watch 6",

        # --- Google ---
        "Google Pixel 8", "Google Pixel 8 Pro", "Google Pixel 8a", "Google Pixel Fold",
        "Google Pixel Tablet", "Google Pixel Buds Pro",

        # --- Xiaomi / Poco ---
        "Xiaomi 14", "Xiaomi 14 Ultra", "Xiaomi 13T Pro", "Redmi Note 13 Pro+", "Poco X6 Pro",

        # --- OnePlus ---
        "OnePlus 12", "OnePlus 12R", "OnePlus Open",

        # --- Motorola ---
        "Motorola Edge 50 Pro", "Motorola Razr 40 Ultra", "Moto G84",

        # --- Інші (Sony, Asus, Nothing) ---
        "Sony Xperia 1 VI", "Sony Xperia 5 V",
        "Asus Zenfone 11 Ultra", "Asus ROG Phone 8 Pro",
        "Nothing Phone (2)", "Nothing Phone (2a)"
    ],
    "price": [
        # Apple Prices
        799, 899, 999, 1199, 1599,
        699, 799, 599, 429,
        999, 599, 999, 1099, 1599,
        249, 549,

        # Samsung Prices
        799, 999, 1299, 1659,
        599, 479, 399,
        1799, 999,
        1199, 299,

        # Google Prices
        699, 999, 499, 1799,
        399, 199,

        # Xiaomi Prices
        999, 1499, 650, 400, 320,

        # OnePlus Prices
        799, 499, 1699,

        # Motorola Prices
        699, 999, 250,

        # Others Prices
        1399, 999,
        899, 1199,
        599, 349
    ]
}

# Перевірка на однакову довжину списків (щоб не було помилок)
assert len(data["item_name"]) == len(data["price"]), "Кількість товарів і цін не співпадає!"

# Збереження
df = pd.DataFrame(data)
df.to_csv("big_tech_catalog.csv", index=False)

print(f"✅ Файл 'big_tech_catalog.csv' створено!")
print(f"📦 В базі даних зараз: {len(df)} товарів.")

✅ Файл 'big_tech_catalog.csv' створено!
📦 В базі даних зараз: 50 товарів.


In [ ]:
import os
import json
import base64
import pandas as pd
import gradio as gr
from email.mime.text import MIMEText
from datetime import datetime, timedelta
from openai import OpenAI
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# --- 1. КОНФІГУРАЦІЯ ---
# os.environ["OPENAI_API_KEY"] = "sk-proj-..."
TOKEN_PATH = '/content/token.json'
PRODUCT_CATALOG = {"iphone 15": 900}

# --- 2. ФУНКЦІЇ ---
def load_catalog_from_csv(file_obj):
    global PRODUCT_CATALOG
    if file_obj is None: return "📂 Файл не обрано"
    try:
        file_path = file_obj.name if hasattr(file_obj, 'name') else file_obj
        df = pd.read_csv(file_path)
        new_catalog = {}
        for _, row in df.iterrows():
            new_catalog[str(row['item_name']).strip().lower()] = int(row['price'])
        PRODUCT_CATALOG = new_catalog
        return f"✅ Завантажено: {len(new_catalog)} товарів"
    except Exception as e: return f"❌ Помилка: {e}"

# --- GOOGLE & TOOLS ---
def get_service(service_name, version):
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH)
        return build(service_name, version, credentials=creds)
    return None

def book_meeting(topic, datetime_str, participants):
    service = get_service('calendar', 'v3')
    if not service: return "Error: Auth failed"
    try:
        start_dt = datetime.fromisoformat(datetime_str)
        event = {
            'summary': topic, 'description': f"Participants: {participants}",
            'start': {'dateTime': start_dt.isoformat(), 'timeZone': 'Europe/Kiev'},
            'end': {'dateTime': (start_dt + timedelta(hours=1)).isoformat(), 'timeZone': 'Europe/Kiev'},
        }
        res = service.events().insert(calendarId='primary', body=event).execute()
        return json.dumps({"status": "success", "link": res.get('htmlLink')})
    except Exception as e: return str(e)

def send_email(recipient, subject, body):
    service = get_service('gmail', 'v1')
    if not service: return "Error: Auth failed"
    try:
        msg = MIMEText(body); msg['to'] = recipient; msg['subject'] = subject
        raw = base64.urlsafe_b64encode(msg.as_bytes()).decode()
        sent = service.users().messages().send(userId='me', body={'raw': raw}).execute()
        return json.dumps({"status": "sent", "id": sent['id']})
    except Exception as e: return str(e)

def get_item_price(item_name):
    global PRODUCT_CATALOG
    key = item_name.lower().strip()
    price = PRODUCT_CATALOG.get(key)
    if price: return json.dumps({"item": item_name, "price": price})
    for k, v in PRODUCT_CATALOG.items():
        if k in key: return json.dumps({"item": k, "price": v})
    return json.dumps({"error": "Not found"})

def calculate_shipping(destination, price):
    return json.dumps({"total": 50 + int(price)*0.05})

available_functions = {
    "book_meeting": book_meeting, "send_email": send_email,
    "get_item_price": get_item_price, "calculate_shipping": calculate_shipping
}

tools = [
    {"type": "function", "function": {"name": "get_item_price", "description": "Check price", "parameters": {"type": "object", "properties": {"item_name": {"type": "string"}}, "required": ["item_name"]}}},
    {"type": "function", "function": {"name": "calculate_shipping", "description": "Calc shipping", "parameters": {"type": "object", "properties": {"destination": {"type": "string"}, "price": {"type": "integer"}}}}},
    {"type": "function", "function": {"name": "book_meeting", "description": "Book meeting (ISO time)", "parameters": {"type": "object", "properties": {"topic": {"type": "string"}, "datetime_str": {"type": "string"}, "participants": {"type": "string"}}}}},
    {"type": "function", "function": {"name": "send_email", "description": "Send email", "parameters": {"type": "object", "properties": {"recipient": {"type": "string"}, "subject": {"type": "string"}, "body": {"type": "string"}}}}}
]

# --- AI CORE ---
client = OpenAI()
# Історія для OpenAI (контекст розмови)
openai_history = [{"role": "system", "content": f"You are a smart enterprise assistant. Time: {datetime.now().strftime('%Y-%m-%d %H:%M')}."}]

def chat_logic(user_input, history):
    # ВАЖЛИВО: history приходить від Gradio. Якщо воно None, ініціалізуємо список.
    if history is None: history = []

    global openai_history
    openai_history.append({"role": "user", "content": user_input})

    logs = []
    bot_response = ""

    try:
        response = client.chat.completions.create(model="gpt-4o-mini", messages=openai_history, tools=tools)
        msg = response.choices[0].message

        if msg.tool_calls:
            openai_history.append(msg)
            for tool in msg.tool_calls:
                func_name = tool.function.name
                args = json.loads(tool.function.arguments)
                logs.append(f"🛠️ CALL: {func_name}\nARGS: {args}")

                func = available_functions.get(func_name)
                if func:
                    if func_name == "get_item_price": res = func(args.get("item_name"))
                    elif func_name == "calculate_shipping": res = func(args.get("destination"), args.get("price"))
                    elif func_name == "book_meeting": res = func(args.get("topic"), args.get("datetime_str"), args.get("participants"))
                    elif func_name == "send_email": res = func(args.get("recipient"), args.get("subject"), args.get("body"))

                    logs.append(f"✅ RESULT: {res}")
                    openai_history.append({"tool_call_id": tool.id, "role": "tool", "name": func_name, "content": str(res)})

            final = client.chat.completions.create(model="gpt-4o-mini", messages=openai_history)
            bot_response = final.choices[0].message.content
        else:
            bot_response = msg.content
            logs.append("ℹ️ No tools needed.")

        openai_history.append({"role": "assistant", "content": bot_response})

    except Exception as e:
        bot_response = f"⚠️ Error: {str(e)}"

    # --- ВИПРАВЛЕННЯ ПОМИЛКИ ---
    # Ми додаємо повідомлення як СЛОВНИКИ, а не списки. Це задовольнить валідатор.
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": bot_response})

    return "", history, "\n\n".join(logs)

# --- UI (ГАРНИЙ ДИЗАЙН, БЕЗ CSS КОНФЛІКТІВ) ---
with gr.Blocks(theme="soft", title="Enterprise Agent") as demo:

    gr.Markdown("# 🤖 Enterprise AI Assistant")

    with gr.Row():
        # --- ЛІВА КОЛОНКА ---
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### ⚙️ Налаштування")
            file_in = gr.File(label="📄 Завантажити базу (CSV)")
            status = gr.Textbox(label="Статус бази", value="Очікування файлу...", interactive=False)

            gr.Markdown("---")
            gr.Markdown("### 🧠 Логи мислення")
            # Акордеон для логів
            with gr.Accordion("Відкрити деталі", open=False):
                logs = gr.TextArea(label="System Logs", lines=15, interactive=False)

        # --- ПРАВА КОЛОНКА ---
        with gr.Column(scale=2):
            # Чатбот без параметрів типу 'type' (щоб не було TypeError)
            chatbot = gr.Chatbot(label="Chat Session", height=550)

            with gr.Row():
                msg = gr.Textbox(
                    show_label=False,
                    placeholder="Введіть запит тут...",
                    scale=4,
                    container=False
                )
                btn = gr.Button("Відправити 🚀", variant="primary", scale=1)

            clear_btn = gr.Button("🗑️ Очистити")

    # Події
    file_in.upload(load_catalog_from_csv, file_in, status)

    btn.click(chat_logic, inputs=[msg, chatbot], outputs=[msg, chatbot, logs])
    msg.submit(chat_logic, inputs=[msg, chatbot], outputs=[msg, chatbot, logs])

    # Очищення (повертаємо порожній список і порожній рядок)
    clear_btn.click(lambda: ([], ""), None, [chatbot, logs], queue=False)

demo.launch(debug=True)

/tmp/ipython-input-168505426.py:141: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme="soft", title="Enterprise Agent") as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e5585937151c1b5883.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://e5585937151c1b5883.gradio.live
